In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

## 1. Data Generation (simulating NASA data for deforestation)


In [2]:
def generate_deforestation_data(n_samples=1000):
    np.random.seed(42)
    
    # Features: temperature, rainfall, human_activity, time
    temperature = np.random.normal(25, 5, n_samples)
    rainfall = np.random.normal(1000, 200, n_samples)
    human_activity = np.random.randint(0, 10, n_samples)
    time = np.linspace(0, 10, n_samples)
    
    # Target: deforestation_rate (simplified model)
    deforestation_rate = (
        0.1 * temperature +
        -0.05 * rainfall +
        0.5 * human_activity +
        0.1 * time +
        np.random.normal(0, 0.1, n_samples)
    )
    deforestation_rate = (deforestation_rate - np.min(deforestation_rate)) / (np.max(deforestation_rate) - np.min(deforestation_rate))
    
    data = pd.DataFrame({
        'temperature': temperature,
        'rainfall': rainfall,
        'human_activity': human_activity,
        'time': time,
        'deforestation_rate': deforestation_rate
    })
    
    return data

## 2. Data Preprocessing


In [3]:
def preprocess_data(data):
    X = data[['temperature', 'rainfall', 'human_activity', 'time']]
    y = data['deforestation_rate']
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test, scaler

## 3. Model Creation


In [4]:
def create_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

## 4. Model Training


In [5]:
def train_model(model, X_train, y_train):
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
    return history

## 5. Quiz Generation


In [6]:
def generate_question(model, scaler, question_type='multiple_choice'):
    # Generate random input
    input_data = np.random.rand(1, 4)
    scaled_input = scaler.transform(input_data)
    
    # Predict deforestation rate
    predicted_rate = model.predict(scaled_input)[0][0]
    
    # Create question based on the prediction
    if question_type == 'multiple_choice':
        question = "Given the following conditions:\n"
        question += f"Temperature: {input_data[0][0]:.2f}°C\n"
        question += f"Rainfall: {input_data[0][1]:.2f} mm\n"
        question += f"Human Activity Level: {input_data[0][2]:.2f}\n"
        question += f"Time Period: {input_data[0][3]:.2f} years\n\n"
        question += "What is the most likely impact on deforestation?\n"
        
        options = [
            f"A) High deforestation rate (>{predicted_rate+0.2:.2f})",
            f"B) Moderate deforestation rate ({predicted_rate-0.1:.2f} - {predicted_rate+0.1:.2f})",
            f"C) Low deforestation rate (<{predicted_rate-0.2:.2f})",
            "D) No significant impact on deforestation"
        ]
        
        correct_answer = 'B'
        
        return question, options, correct_answer
    
    elif question_type == 'true_false':
        statement = "Given the environmental conditions, "
        if predicted_rate > 0.5:
            statement += "the deforestation rate is likely to be high."
            correct_answer = True
        else:
            statement += "the deforestation rate is likely to be low."
            correct_answer = False
        
        return statement, correct_answer

## 6. Main Execution


In [7]:
if __name__ == "__main__":
    # Generate and preprocess data
    data = generate_deforestation_data()
    X_train, X_test, y_train, y_test, scaler = preprocess_data(data)
    
    # Create and train model
    model = create_model((X_train.shape[1],))
    history = train_model(model, X_train, y_train)
    
    # Evaluate model
    test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test MAE: {test_mae:.4f}")
    
    # Generate questions
    print("\nMultiple Choice Question:")
    mc_question, mc_options, mc_answer = generate_question(model, scaler, 'multiple_choice')
    print(mc_question)
    for option in mc_options:
        print(option)
    print(f"Correct Answer: {mc_answer}")
    
    print("\nTrue/False Question:")
    tf_statement, tf_answer = generate_question(model, scaler, 'true_false')
    print(tf_statement)
    print(f"Correct Answer: {tf_answer}")


D:\NASA\ButterflyEarth\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test MAE: 0.0058

Multiple Choice Question:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Given the following conditions:
Temperature: 0.85°C
Rainfall: 0.18 mm
Human Activity Level: 0.96
Time Period: 0.38 years

What is the most likely impact on deforestation?

A) High deforestation rate (>1.17)
B) Moderate deforestation rate (0.87 - 1.07)
C) Low deforestation rate (<0.77)
D) No significant impact on deforestation
Correct Answer: B

True/False Question:


D:\NASA\ButterflyEarth\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\NASA\ButterflyEarth\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Given the environmental conditions, the deforestation rate is likely to be high.
Correct Answer: True


## 7. Interactive Quiz Function


In [8]:
def take_quiz(model, scaler, num_questions=5):
    score = 0
    for i in range(num_questions):
        print(f"\nQuestion {i+1}:")
        if np.random.rand() > 0.5:
            # Multiple Choice Question
            question, options, correct_answer = generate_question(model, scaler, 'multiple_choice')
            print(question)
            for option in options:
                print(option)
            user_answer = input("Your answer (A/B/C/D): ").upper()
            if user_answer == correct_answer:
                print("Correct!")
                score += 1
            else:
                print(f"Incorrect. The correct answer is {correct_answer}.")
        else:
            # True/False Question
            statement, correct_answer = generate_question(model, scaler, 'true_false')
            print(statement)
            user_answer = input("True or False? (T/F): ").upper() == 'T'
            if user_answer == correct_answer:
                print("Correct!")
                score += 1
            else:
                print(f"Incorrect. The correct answer is {correct_answer}.")
    
    print(f"\nQuiz completed! Your score: {score}/{num_questions}")

## 8. Run the quiz


In [9]:
if __name__ == "__main__":
    print("\nWelcome to the Deforestation Chapter!")
    take_quiz(model, scaler)


Welcome to the Deforestation Chapter!

Question 1:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


D:\NASA\ButterflyEarth\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Given the environmental conditions, the deforestation rate is likely to be high.
Correct!

Question 2:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


D:\NASA\ButterflyEarth\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Given the following conditions:
Temperature: 0.86°C
Rainfall: 0.76 mm
Human Activity Level: 0.15
Time Period: 0.69 years

What is the most likely impact on deforestation?

A) High deforestation rate (>1.17)
B) Moderate deforestation rate (0.87 - 1.07)
C) Low deforestation rate (<0.77)
D) No significant impact on deforestation
Incorrect. The correct answer is B.

Question 3:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Given the following conditions:
Temperature: 0.68°C
Rainfall: 0.33 mm
Human Activity Level: 0.50
Time Period: 0.65 years

What is the most likely impact on deforestation?


...


A) High deforestation rate (>1.17)
B) Moderate deforestation rate (0.87 - 1.07)
C) Low deforestation rate (<0.77)
D) No significant impact on deforestation
Incorrect. The correct answer is B.

Question 4:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step


D:\NASA\ButterflyEarth\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Given the following conditions:
Temperature: 0.12°C
Rainfall: 0.31 mm
Human Activity Level: 0.80
Time Period: 0.86 years

What is the most likely impact on deforestation?

A) High deforestation rate (>1.17)
B) Moderate deforestation rate (0.87 - 1.07)
C) Low deforestation rate (<0.77)
D) No significant impact on deforestation
Incorrect. The correct answer is B.

Question 5:


D:\NASA\ButterflyEarth\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
Given the environmental conditions, the deforestation rate is likely to be high.
Incorrect. The correct answer is True.

Quiz completed! Your score: 1/5


In [12]:
import joblib
from sklearn.ensemble import RandomForestClassifier

# Create a scikit-learn model
model = RandomForestClassifier()

# Train the model on your data

# Save the model to a file
joblib.dump(model, '1-Deforestation.pkl')

['1-Deforestation.pkl']